In [29]:
import pandas as pd
from numpy import nan
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)

# Home Data pipeline

See [readme](../README.md) 2do section for next steps

In [30]:
# load data

global training___data, validation_data, training___price, validation_price
def load_data():
    global training___data, validation_data, training___price, validation_price
    data = pd.read_csv("./train.csv")
    training___data, validation_data, training___price, validation_price = train_test_split(data, data.SalePrice, random_state = 0)

load_data()
training___data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1292,1293,70,RM,60.0,6600,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,5,4,1892,1965,Gable,CompShg,Wd Sdng,Wd Sdng,NaN,0.0,TA,TA,Stone,TA,TA,No,Unf,0,Unf,0,994,994,GasA,TA,N,SBrkr,1378,994,0,2372,0,0,2,0,4,2,TA,11,Min2,0,NaN,Attchd,1985.0,RFn,1,432,TA,TA,Y,0,287,0,0,0,0,NaN,NaN,NaN,0,12,2009,WD,Normal,107500
1018,1019,80,RL,NaN,10784,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,Gilbert,Norm,Norm,1Fam,SLvl,7,5,1991,1992,Gable,CompShg,HdBoard,HdBoard,BrkFace,76.0,Gd,TA,PConc,Gd,TA,No,Unf,0,Unf,0,384,384,GasA,Gd,Y,SBrkr,802,670,0,1472,0,0,2,1,3,1,Gd,7,Typ,1,TA,Attchd,1991.0,RFn,2,402,TA,TA,Y,164,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,160000
1213,1214,80,RL,NaN,10246,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,Norm,Norm,1Fam,SLvl,4,9,1965,2001,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,Gd,CBlock,TA,Gd,Av,GLQ,648,Unf,0,0,648,GasA,Ex,Y,SBrkr,960,0,0,960,1,1,0,0,0,1,TA,3,Typ,0,NaN,Attchd,1965.0,Unf,1,364,TA,TA,Y,88,0,0,0,0,0,NaN,NaN,NaN,0,5,2006,WD,Normal,145000
1430,1431,60,RL,60.0,21930,Pave,NaN,IR3,Lvl,AllPub,Inside,Gtl,Gilbert,RRAn,Norm,1Fam,2Story,5,5,2005,2005,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,Gd,TA,PConc,Gd,Gd,Av,Unf,0,Unf,0,732,732,GasA,Ex,Y,SBrkr,734,1104,0,1838,0,0,2,1,4,1,TA,7,Typ,1,Gd,BuiltIn,2005.0,Fin,2,372,TA,TA,Y,100,40,0,0,0,0,NaN,NaN,NaN,0,7,2006,WD,Normal,192140
810,811,20,RL,78.0,10140,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1974,1999,Hip,CompShg,HdBoard,HdBoard,BrkFace,99.0,TA,TA,CBlock,TA,TA,No,ALQ,663,LwQ,377,0,1040,GasA,Fa,Y,SBrkr,1309,0,0,1309,1,0,1,1,3,1,Gd,5,Typ,1,Fa,Attchd,1974.0,RFn,2,484,TA,TA,Y,265,0,0,0,0,648,Fa,GdPrv,NaN,0,1,2006,WD,Normal,181000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,764,60,RL,82.0,9430,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,1999,1999,Gable,CompShg,VinylSd,VinylSd,BrkFace,673.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,1163,Unf,0,89,1252,GasA,Ex,Y,SBrkr,1268,1097,0,2365,1,0,2,1,3,1,Gd,8,Typ,1,Gd,Attchd,1999.0,RFn,3,856,TA,TA,Y,0,128,0,0,180,0,NaN,NaN,NaN,0,7,2009,WD,Normal,337000
835,836,20,RL,60.0,9600,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,4,7,1950,1995,Gable,CompShg,VinylSd,HdBoard,NaN,0.0,TA,TA,CBlock,Gd,TA,No,BLQ,442,Unf,0,625,1067,GasA,TA,Y,SBrkr,1067,0,0,1067,0,0,2,0,2,1,Gd,4,Min2,0,NaN,Attchd,1996.0,Unf,2,436,TA,TA,Y,290,0,0,0,0,0,NaN,NaN,NaN,0,2,2010,WD,Normal,128000
1216,1217,90,RM,68.0,8930,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,RRAe,Norm,Duplex,1.5Fin,6,5,1978,1978,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,Slab,NaN,NaN,NaN,NaN,0,NaN,0,0,0,GasA,TA,Y,SBrkr,1318,584,0,1902,0,0,2,0,4,2,TA,8,Typ,0,NaN,Attchd,1978.0,Unf,2,539,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,112000
559,560,120,RL,NaN,3196,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,TwnhsE,1Story,7,5,2003,2004,Gable,CompShg,VinylSd,VinylSd,BrkFace,18.0,Gd,TA,PConc,Gd,TA,Gd,Unf,0,Unf,0,1374,1374,G

In [31]:
# define possible column transformations

is_nan = lambda x: x != x
equals = lambda a, b: (is_nan(a) and is_nan(b)) or a == b
all_unique_values = lambda column_name: set(training___data[column_name].unique()) | set(validation_data[column_name].unique())

def drop(column_name, data_set, *_):
    data_set.drop(column_name, axis=1, inplace=True)
    return data_set

def drop_row(column_name, data_set, condition):
    return data_set[~data_set[column_name].apply(condition)]

def one_hot(column_name, data_set, options):
    keys = options if options else data_set[column_name].unique()
    for key in keys:
        data_set[f"{column_name}_{key}"] = data_set[column_name].apply(lambda x: 1 if equals(x, key) else 0)
    return drop(column_name, data_set)

def re_map(column_name, data_set, mapping):
    data_set[column_name] = data_set[column_name].apply(lambda x: mapping[x])
    return data_set

def MSZoning(column_name, data_set, *_):
    options = dict(
        # A="Agriculture",
        C="Commercial",
        FV="Floating Village Residential",
        # I="Industrial",
        RH="Residential High Density",
        RL="Residential Low Density",
        # RP="Residential Low Density Park",
        RM="Residential Medium Density",
    )
    for key, name in options.items():
        if key == "C":
            continue
        data_set[f"zoned_{name}"] = data_set[column_name].apply(lambda x: 1 if x in [key, "C"] else 0)
    return drop(column_name, data_set)

def masonry(column_name, data_set, *_):
    options = dict(
        BrkCmn="Brick Common",
        BrkFace="Brick Face",
        CBlock="Cinder Block",
        Stone="Stone",
    )
    for key in options:
        data_set[f"masonry {options[key]}"] = 0
        for id in range(len(data_set)):
            if data_set[column_name].iloc[id] == key:
                data_set[f"masonry {options[key]}"].iat[id] = data_set["MasVnrArea"].iloc[id]
    
    data_set.drop(["MasVnrType", "MasVnrArea"], axis=1, inplace=True)
    return data_set

def basement(column_name, data_set, *_):
    options = dict(
        GLQ="Good Living Quarters",
        ALQ="Average Living Quarters",
        BLQ="Below Average Living Quarters",
        Rec="Average Rec Room",
        LwQ="Low Quality",
        Unf="Unfinished",
    )
    for key in options:
        data_set[f"basement {options[key]} sqft"] = 0
        for id in range(len(data_set)):
            if data_set["BsmtFinType1"].iloc[id] == key:
                data_set[f"basement {options[key]} sqft"].iat[id] += data_set["BsmtFinSF1"].iloc[id]
            if data_set["BsmtFinType2"].iloc[id] == key:
                data_set[f"basement {options[key]} sqft"].iat[id] += data_set["BsmtFinSF2"].iloc[id]
    
    data_set.drop(["BsmtFinType1", "BsmtFinSF1", "BsmtFinType2", "BsmtFinSF2"], axis=1, inplace=True)
    return data_set

def basement_unfinished(column_name, data_set, *_):
    unf1 = "basement Unfinished sqft"
    unf2 = "BsmtUnfSF"
    data_set[unf1] = data_set.apply(lambda x: x[unf1] + x[unf2], axis=1)
    
    data_set.drop(["BsmtUnfSF"], axis=1, inplace=True)
    return data_set

import inflation
def sale_price(column_name, data_set, *_):
    for id in range(len(data_set)):
        data_set[column_name].iat[id] = inflation.adjust(
            price = data_set[column_name].iloc[id],
            year = data_set["YrSold"].iloc[id],
            month = data_set["MoSold"].iloc[id],
        )
    data_set.drop(["MoSold", "YrSold"], axis=1, inplace=True)
    return data_set

In [32]:
# specify each column transformation

transformations = {
    "MSSubClass": (drop, None),
    "MSZoning": (MSZoning, None),
    "LotFrontage": (drop_row, is_nan),
    "Street": (re_map, {"Pave": 1, "Grvl": 0}),
    "Alley": (re_map, {"Pave": 2, "Grvl": 1, nan: 0}),
    "LotShape": (re_map, {"Reg": 0, "IR1": 1, "IR2": 2, "IR3": 3}),
    "LandContour": (re_map, {"Lvl": 1, "Bnk": 3, "HLS": 2, "Low": 0}),
    "Utilities": (drop, None),
    "LotConfig": (one_hot, all_unique_values("LotConfig")),
    "LandSlope": (re_map, {"Gtl": 0, "Mod": 1, "Sev": 2}),
    "Neighborhood": (one_hot, all_unique_values("Neighborhood")),
    "Condition1": (drop, None),
    "Condition2": (drop, None),
    "BldgType": (one_hot, all_unique_values("BldgType")),
    "YearBuilt": (drop, None),
    "RoofStyle": (one_hot, all_unique_values("RoofStyle")),
    "RoofMatl": (one_hot, all_unique_values("RoofMatl")),
    "Exterior1st": (one_hot, all_unique_values("Exterior1st")),
    "Exterior2nd": (drop, None),
    "MasVnrType": (masonry, None),
    "ExterQual": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0}),
    "ExterCond": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0}),
    "Foundation": (one_hot, all_unique_values("Foundation")),
    "BsmtQual": (re_map, {"Ex": 110, "Gd": 95, "TA": 85, "Fa": 75, "Po": 60, "NA": 0, nan: 0}),
    # "BsmtCond": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0, "NA": 2, nan: 2}),
    "BsmtCond": (drop, None),
    "BsmtExposure": (re_map, {"Gd": 3, "Av": 2, "Mn": 1, "No": 0, "NA": 0, nan: 0}),
    "TotalBsmtSF": (basement, None),
    "BsmtUnfSF": (basement_unfinished, None),
    "Heating": (one_hot, all_unique_values("Heating")),
    "HeatingQC": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0}),
    "CentralAir": (re_map, {"Y": 1, "N": 0}),
    "Electrical": (re_map, {"SBrkr": 4, "FuseA": 3, "FuseF": 2, "FuseP": 1, "Mix": 0, nan: 0}),
    "KitchenQual": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0}),
    "Functional": (re_map, {"Typ": 7, "Min1": 6, "Min2": 5, "Mod": 4, "Maj1": 3, "Maj2": 2, "Sev": 1, "Sal": 0}),
    "FireplaceQu": (re_map, {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, nan: 3}),
    # TODO: handle everything to do with garages
    "PavedDrive": (re_map, {"Y": 2, "P": 1, "N": 0}),
    "PoolQC": (re_map, {"Ex": 5, "Gd": 4, "TA": 2, "Fa": 1, nan: 3}),
    "Fence": (one_hot, all_unique_values("Fence")),
    "MiscFeature": (drop, None),
    "SaleType": (one_hot, all_unique_values("SaleType")),
    "SaleCondition": (one_hot, all_unique_values("SaleCondition")),
    "SalePrice": (sale_price, None),
}

In [33]:
# apply column transformations to data
load_data()
for column_name, (transform, option) in transformations.items():
    print("transforming", column_name)
    training___data = transform(column_name, training___data, option)
    validation_data = transform(column_name, validation_data, option)

validation_data

transforming MSSubClass
transforming MSZoning
transforming LotFrontage
transforming Street
transforming Alley
transforming LotShape
transforming LandContour
transforming Utilities
transforming LotConfig
transforming LandSlope
transforming Neighborhood
transforming Condition1
transforming Condition2
transforming BldgType
transforming YearBuilt
transforming RoofStyle
transforming RoofMatl
transforming Exterior1st
transforming Exterior2nd
transforming MasVnrType
transforming ExterQual
transforming ExterCond
transforming Foundation
transforming BsmtQual
transforming BsmtCond
transforming BsmtExposure
transforming TotalBsmtSF
transforming BsmtUnfSF
transforming Heating
transforming HeatingQC
transforming CentralAir
transforming Electrical
transforming KitchenQual
transforming Functional
transforming FireplaceQu
transforming PavedDrive
transforming PoolQC
transforming Fence
transforming MiscFeature
transforming SaleType
transforming SaleCondition
transforming SalePrice


/tmp/ipykernel_13546/1106347058.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_set[f"{column_name}_{key}"] = data_set[column_name].apply(lambda x: 1 if equals(x, key) else 0)
/tmp/ipykernel_13546/1106347058.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_set[f"{column_name}_{key}"] = data_set[column_name].apply(lambda x: 1 if equals(x, key) else 0)
/tmp/ipykernel_13546/1106347058.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which

,Id,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LandSlope,HouseStyle,OverallQual,OverallCond,YearRemodAdd,ExterQual,ExterCond,BsmtQual,BsmtExposure,TotalBsmtSF,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,SalePrice,zoned_Floating Village Residential,zoned_Residential High Density,zoned_Residential Low Density,zoned_Residential Medium Density,LotConfig_Inside,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Corner,Neighborhood_BrkSide,Neighborhood_Gilbert,Neighborhood_Mitchel,Neighborhood_StoneBr,Neighborhood_BrDale,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_Somerst,Neighborhood_Timber,Neighborhood_Crawfor,Neighborhood_SWISU,Neighborhood_NWAmes,Neighborhood_OldTown,Neighborhood_NAmes,Neighborhood_Edwards,Neighborhood_CollgCr,Neighborhood_Sawyer,Neighborhood_IDOTRR,Neighborhood_Veenker,Neighborhood_SawyerW,Neighborhood_NoRidge,Neighborhood_MeadowV,Neighborhood_NridgHt,Neighborhood_ClearCr,Neighborhood_NPkVill,BldgType_1Fam,BldgType_Twnhs,BldgType_Duplex,BldgType_TwnhsE,BldgType_2fmCon,RoofStyle_Mansard,RoofStyle_Hip,RoofStyle_Flat,RoofStyle_Shed,RoofStyle_Gable,RoofStyle_Gambrel,RoofMatl_Membran,RoofMatl_Tar&Grv,RoofMatl_CompShg,RoofMatl_WdShngl,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_WdShake,RoofMatl_ClyTile,Exterior1st_BrkComm,Exterior1st_MetalSd,Exterior1st_Wd Sdng,Exterior1st_HdBoard,Exterior1st_AsphShn,Exterior1st_Stone,Exterior1st_BrkFace,Exterior1st_Plywood,Exterior1st_AsbShng,Exterior1st_VinylSd,Exterior1st_WdShing,Exterior1st_Stucco,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_ImStucc,masonry Brick Common,masonry Brick Face,masonry Cinder Block,masonry Stone,Foundation_Stone,Foundation_CBlock,Foundation_BrkTil,Foundation_Slab,Foundation_PConc,Foundation_Wood,basement Good Living Quarters sqft,basement Average Living Quarters sqft,basement Below Average Living Quarters sqft,basement Average Rec Room sqft,basement Low Quality sqft,basement Unfinished sqft,Heating_GasA,Heating_Grav,Heating_Wall,Heating_Floor,Heating_OthW,Heating_GasW,Fence_GdWo,Fence_GdPrv,Fence_MnPrv,Fence_MnWw,Fence_nan,SaleType_New,SaleType_ConLI,SaleType_CWD,SaleType_ConLw,SaleType_COD,SaleType_ConLD,SaleType_Con,SaleType_WD,SaleType_Oth,SaleCondition_Alloca,SaleCondition_Abnorml,SaleCondition_Partial,SaleCondition_Normal,SaleCondition_AdjLand,SaleCondition_Family
491,492,79.0,9490,1,0,0,1,0,1.5Fin,6,7,1950,2,2,85,0,806,2,1,3,958,620,0,1578,1,0,1,0,3,1,1,5,7,2,3,Attchd,1941.0,Unf,1,240,TA,TA,2,0,0,32,0,0,0,3,0,141479.052823,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,403,165,0,238,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
279,280,83.0,10005,1,0,0,1,0,2Story,7,5,1977,2,2,95,0,1160,4,1,4,1156,866,0,2022,0,0,2,1,4,1,2,8,7,1,3,Attchd,1977.0,Fin,2,505,TA,TA,2,288,117,0,0,0,0,3,0,199161.634103,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,299,0,0,0,1,0,0,0,0,0,0,392,0,0,768,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
655,656,21.0,1680,1,0,0,1,0,2Story,6,5,1971,2,2,85,0,525,2,1,4,525,567,0,1092,0,0,1,1,3,1,2,6,7,0,3,Detchd,1971.0,Unf,1,264,TA,TA,2,0,0,0,0,0,0,3,0,88913.494810,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,381,0,0,0,1,0,0,0,0,0,0,0,0,0,525,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1013,1014,60.0,7200,1,0,0,1,0,1Story,5,4,2006,2,2,85,0,1022,2,0,4,1022,0,0,1022,1,0,1,0,2,1,2,4,2,0,3,Detchd,1956.0,Unf,1,280,TA,TA,2,0,30,226,0,0,0,3,0,86255.430061,0,0,0,1,1,0,0,0,0,0,0,

In [34]:
training___data.describe()
validation_data.describe()

,Id,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LandSlope,OverallQual,OverallCond,YearRemodAdd,ExterQual,ExterCond,BsmtQual,BsmtExposure,TotalBsmtSF,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageCars,GarageArea,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,SalePrice,zoned_Floating Village Residential,zoned_Residential High Density,zoned_Residential Low Density,zoned_Residential Medium Density,LotConfig_Inside,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Corner,Neighborhood_BrkSide,Neighborhood_Gilbert,Neighborhood_Mitchel,Neighborhood_StoneBr,Neighborhood_BrDale,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_Somerst,Neighborhood_Timber,Neighborhood_Crawfor,Neighborhood_SWISU,Neighborhood_NWAmes,Neighborhood_OldTown,Neighborhood_NAmes,Neighborhood_Edwards,Neighborhood_CollgCr,Neighborhood_Sawyer,Neighborhood_IDOTRR,Neighborhood_Veenker,Neighborhood_SawyerW,Neighborhood_NoRidge,Neighborhood_MeadowV,Neighborhood_NridgHt,Neighborhood_ClearCr,Neighborhood_NPkVill,BldgType_1Fam,BldgType_Twnhs,BldgType_Duplex,BldgType_TwnhsE,BldgType_2fmCon,RoofStyle_Mansard,RoofStyle_Hip,RoofStyle_Flat,RoofStyle_Shed,RoofStyle_Gable,RoofStyle_Gambrel,RoofMatl_Membran,RoofMatl_Tar&Grv,RoofMatl_CompShg,RoofMatl_WdShngl,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_WdShake,RoofMatl_ClyTile,Exterior1st_BrkComm,Exterior1st_MetalSd,Exterior1st_Wd Sdng,Exterior1st_HdBoard,Exterior1st_AsphShn,Exterior1st_Stone,Exterior1st_BrkFace,Exterior1st_Plywood,Exterior1st_AsbShng,Exterior1st_VinylSd,Exterior1st_WdShing,Exterior1st_Stucco,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_ImStucc,masonry Brick Common,masonry Brick Face,masonry Cinder Block,masonry Stone,Foundation_Stone,Foundation_CBlock,Foundation_BrkTil,Foundation_Slab,Foundation_PConc,Foundation_Wood,basement Good Living Quarters sqft,basement Average Living Quarters sqft,basement Below Average Living Quarters sqft,basement Average Rec Room sqft,basement Low Quality sqft,basement Unfinished sqft,Heating_GasA,Heating_Grav,Heating_Wall,Heating_Floor,Heating_OthW,Heating_GasW,Fence_GdWo,Fence_GdPrv,Fence_MnPrv,Fence_MnWw,Fence_nan,SaleType_New,SaleType_ConLI,SaleType_CWD,SaleType_ConLw,SaleType_COD,SaleType_ConLD,SaleType_Con,SaleType_WD,SaleType_Oth,SaleCondition_Alloca,SaleCondition_Abnorml,SaleCondition_Partial,SaleCondition_Normal,SaleCondition_AdjLand,SaleCondition_Family
count,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.00000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,282.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.00000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.0,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.00000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.0,305.000000,305.000000,305.0,305.000000,305.000000,305.000000,305.0,305.0,305.0,305.000000,305.000000,305.000000,305.000000,305.000000,305.0,305.0,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.0,305.000000,305.0,305.000000,305.000000,305.0,305.000000,305.0,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.00000,305.0,305.000000,305